In [ ]:
# Cell 2 - Code
# Cài đặt các thư viện cần thiết
!pip install faster-whisper tqdm ffmpeg-python pydub pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00


In [ ]:
# Cell 3 - Code
from faster_whisper import WhisperModel
import logging
from tqdm import tqdm
import ffmpeg
import os
import pandas as pd
from pydub import AudioSegment
from google.colab import drive
import re

# Thiết lập logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [ ]:
# Cell 4 - Code
# Kết nối với Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cell 5 - Code
def get_audio_duration(audio_path):
    """
    Lấy độ dài của file âm thanh
    """
    try:
        probe = ffmpeg.probe(audio_path)
        duration = float(probe['streams'][0]['duration'])
        logging.info(f"Độ dài file âm thanh: {duration:.2f} giây")
        return duration
    except Exception as e:
        logging.error(f"Lỗi khi lấy độ dài file âm thanh: {str(e)}")
        return None


In [ ]:
# Cell 6 - Code
def parse_timestamp(timestamp_str):
    """
    Chuyển đổi timestamp từ định dạng [start -> end] thành số giây
    """
    try:
        # Tìm các số trong chuỗi timestamp
        numbers = re.findall(r'\d+\.?\d*', timestamp_str)
        if len(numbers) >= 2:
            return float(numbers[0]), float(numbers[1])
    except Exception as e:
        logging.error(f"Lỗi khi parse timestamp {timestamp_str}: {str(e)}")
    return None, None

In [ ]:
# Cell 7 - Code
def split_audio_by_transcript(audio_path, transcript_path, output_dir, min_segment_length=0.5, max_segment_length=30.0):
    """
    Cắt file âm thanh theo transcript

    Args:
        audio_path: Đường dẫn đến file âm thanh
        transcript_path: Đường dẫn đến file transcript
        output_dir: Thư mục lưu các file âm thanh đã cắt
        min_segment_length: Độ dài tối thiểu của đoạn (giây)
        max_segment_length: Độ dài tối đa của đoạn (giây)
    """
    try:
        # Tạo thư mục output nếu chưa tồn tại
        os.makedirs(output_dir, exist_ok=True)

        # Đọc file transcript
        with open(transcript_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        # Đọc file âm thanh
        audio = AudioSegment.from_file(audio_path)

        # Tạo DataFrame để lưu thông tin các đoạn
        segments_data = []

        # Xử lý từng dòng trong transcript
        for i, line in enumerate(lines):
            line = line.strip()
            if not line:
                continue

            # Tìm timestamp trong dòng
            timestamp_match = re.search(r'\[(\d+\.?\d*)s -> (\d+\.?\d*)s\]', line)
            if not timestamp_match:
                continue

            start_time = float(timestamp_match.group(1))
            end_time = float(timestamp_match.group(2))
            text = line[line.find(']') + 1:].strip()

            # Kiểm tra độ dài đoạn
            segment_length = end_time - start_time
            if segment_length < min_segment_length or segment_length > max_segment_length:
                logging.debug(f"Bỏ qua đoạn quá ngắn hoặc quá dài: [{start_time:.2f}s -> {end_time:.2f}s]")
                continue

            # Chuyển đổi thời gian từ giây sang mili giây
            start_ms = int(start_time * 1000)
            end_ms = int(end_time * 1000)

            # Cắt đoạn âm thanh
            segment = audio[start_ms:end_ms]

            # Tạo tên file output
            output_filename = f"segment_{i+1:03d}.wav"
            output_path = os.path.join(output_dir, output_filename)

            # Lưu file âm thanh
            segment.export(output_path, format="wav")

            # Thêm thông tin vào DataFrame
            segments_data.append({
                'segment_id': i+1,
                'start_time': start_time,
                'end_time': end_time,
                'duration': segment_length,
                'text': text,
                'audio_file': output_filename
            })

        # Lưu DataFrame vào file CSV
        df = pd.DataFrame(segments_data)
        csv_path = os.path.join(transcript_path, 'segments_info.csv')
        df.to_csv(csv_path, sep='|', index=False, encoding='utf-8')

        logging.info(f"Đã cắt {len(segments_data)} đoạn âm thanh và lưu vào thư mục {transcript_path}")
        logging.info(f"Thông tin chi tiết được lưu vào file: {csv_path}")

    except Exception as e:
        logging.error(f"Lỗi khi cắt file âm thanh: {str(e)}")
        raise


In [ ]:
# Cell 8 - Code
def transcribe_audio(audio_path, output_path=None, min_segment_length=0.5, max_segment_length=30.0):
    """
    Transcribe audio file và bỏ qua các đoạn không có giọng nói

    Args:
        audio_path: Đường dẫn đến file âm thanh
        output_path: Đường dẫn file output (tùy chọn)
        min_segment_length: Độ dài tối thiểu của đoạn (giây)
        max_segment_length: Độ dài tối đa của đoạn (giây)
    """
    # Lấy độ dài file âm thanh
    duration = get_audio_duration(audio_path)
    if duration is None:
        logging.warning("Không thể lấy độ dài file âm thanh, sẽ không hiển thị tiến độ chính xác")
        duration = 0

    # Khởi tạo model Whisper
    logging.info("Đang khởi tạo model Whisper...")
    model = WhisperModel("large-v3", device="cuda", compute_type="float16")

    # Thực hiện transcribe
    logging.info("Bắt đầu quá trình transcribe...")
    segments, info = model.transcribe(audio_path, language="vi", beam_size=5)

    # In thông tin về ngôn ngữ được phát hiện
    logging.info(f"Detected language: {info.language} (probability: {info.language_probability:.2f})")

    # Tạo thanh tiến độ
    pbar = tqdm(total=duration, unit="s", desc="Transcribing")
    last_time = 0

    # Xử lý kết quả
    if output_path:
        with open(output_path, "w", encoding="utf-8") as f:
            for segment in segments:
                # Cập nhật tiến độ
                current_time = segment.end
                pbar.update(current_time - last_time)
                last_time = current_time

                # Kiểm tra độ dài đoạn và nội dung
                segment_length = segment.end - segment.start
                text = segment.text.strip()

                # Bỏ qua nếu:
                # 1. Đoạn quá ngắn (có thể là tiếng ồn)
                # 2. Đoạn quá dài (có thể là đoạn im lặng)
                # 3. Không có nội dung hoặc chỉ có ký tự đặc biệt
                if (segment_length < min_segment_length or
                    segment_length > max_segment_length or
                    not text or
                    text.isspace() or
                    all(not c.isalnum() for c in text)):
                    logging.debug(f"Bỏ qua đoạn: [{segment.start:.2f}s -> {segment.end:.2f}s] {text}")
                    continue

                f.write(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {text}\n")
        pbar.close()
        logging.info(f"Transcript đã được lưu vào file: {output_path}")
    else:
        # In kết quả ra màn hình
        for segment in segments:
            # Cập nhật tiến độ
            current_time = segment.end
            pbar.update(current_time - last_time)
            last_time = current_time

            # Kiểm tra độ dài đoạn và nội dung
            segment_length = segment.end - segment.start
            text = segment.text.strip()

            # Bỏ qua nếu:
            # 1. Đoạn quá ngắn (có thể là tiếng ồn)
            # 2. Đoạn quá dài (có thể là đoạn im lặng)
            # 3. Không có nội dung hoặc chỉ có ký tự đặc biệt
            if (segment_length < min_segment_length or
                segment_length > max_segment_length or
                not text or
                text.isspace() or
                all(not c.isalnum() for c in text)):
                logging.debug(f"Bỏ qua đoạn: [{segment.start:.2f}s -> {segment.end:.2f}s] {text}")
                continue

            logging.info(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {text}")
        pbar.close()

In [ ]:
# Cell 10 - Code
# Đường dẫn tới file trong Google Drive
DRIVE_BASE_PATH = '/content/drive/MyDrive/AI/AudioTranscription'  # Thay đổi nếu cần
AUDIO_FILE_PATH = os.path.join(DRIVE_BASE_PATH, 'audio.wav')  # Đường dẫn tới file âm thanh
OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'whisper_output')  # Thư mục lưu kết quả

In [ ]:
# Cell 12 - Code
# Tạo tên file output
transcript_path = os.path.join(OUTPUT_DIR, "transcript.txt")


# Chạy transcribe
transcribe_audio(AUDIO_FILE_PATH, transcript_path)



Transcribing:   0%|          | 0/3270.528776 [00:00<?, ?s/s]
Transcribing:   0%|          | 4.94/3270.528776 [00:05<1:00:52,  1.12s/s]
Transcribing:   1%|          | 33.42/3270.528776 [00:08<12:09,  4.44s/s] 
Transcribing:   2%|▏         | 63.62/3270.528776 [00:12<08:52,  6.03s/s]
Transcribing:   3%|▎         | 90.84/3270.528776 [00:17<09:12,  5.75s/s]
Transcribing:   4%|▍         | 122.66000000000001/3270.528776 [00:21<07:58,  6.58s/s]
Transcribing:   5%|▍         | 151.16/3270.528776 [00:25<07:36,  6.84s/s]            
Transcribing:   6%|▌         | 184.48/3270.528776 [00:28<06:48,  7.55s/s]
Transcribing:   6%|▋         | 210.84/3270.528776 [00:34<07:54,  6.44s/s]
Transcribing:   7%|▋         | 238.94/3270.528776 [00:38<07:49,  6.45s/s]
Transcribing:   8%|▊         | 270.9/3270.528776 [00:42<07:16,  6.87s/s] 
Transcribing:   9%|▉         | 299.90000000000003/3270.528776 [00:47<07:13,  6.85s/s]
Transcribing:  10%|█         | 327.90000000000003/3270.528776 [00:51<07:25,  6.61s/s]
Tran

In [ ]:

# Cell 14 - Code
# Tạo thư mục cho các đoạn âm thanh
segments_dir = os.path.join(OUTPUT_DIR, "segments")

# Cắt file âm thanh
split_audio_by_transcript(AUDIO_FILE_PATH, transcript_path, segments_dir)

In [ ]:
# Cell 16 - Code
# Đọc file CSV
csv_path = os.path.join(segments_dir, 'segments_info.csv')
df = pd.read_csv(csv_path)
print("Thông tin các đoạn âm thanh:")
print(df)

Thông tin các đoạn âm thanh:
      segment_id  start_time  end_time  duration  \
0              1        0.00      4.94      4.94   
1              2        5.78     11.46      5.68   
2              3       12.50     15.24      2.74   
3              4       16.10     18.56      2.46   
4              5       19.70     24.12      4.42   
...          ...         ...       ...       ...   
2463        2464     3264.96   3266.34      1.38   
2464        2465     3266.34   3267.32      0.98   
2465        2466     3267.32   3268.20      0.88   
2466        2467     3268.20   3268.80      0.60   
2467        2468     3268.80   3269.74      0.94   

                                                   text        audio_file  
0     Thế giới là một nơi điên rồ, và cách mà nó đượ...   segment_001.wav  
1     Nhưng chúng ta phải đủ điên để nhận ra cách th...   segment_002.wav  
2     Nó không chỉ là chuyện của những công thức đẹp...   segment_003.wav  
3     có những ý tưởng ngoài kia đang chờ 